In [1]:
############################load img############################################
# load the val annotations file

import os

def get_annotations_map():
    valAnnotationsPath = './tiny-imagenet-200/val/val_annotations.txt'
    valAnnotationsFile = open(valAnnotationsPath, 'r')
    valAnnotationsContents = valAnnotationsFile.read()
    valAnnotations = {}

    for line in valAnnotationsContents.splitlines():
        pieces = line.strip().split()
        valAnnotations[pieces[0]] = pieces[1]
    
    return valAnnotations


#Sys
import numpy as np
from PIL import Image


def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,64,64],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    #print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,64,64],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


   # print('finished loading test images')+str(i)

    return X_train,y_train,X_test,y_test

#########################################################################################

if __name__ == "__main__":
    
    #Keras
    import keras
    from keras.datasets import mnist
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras.optimizers import RMSprop
    from keras.layers.core import Dense, Dropout, Activation, Flatten
    from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
    from keras.layers.normalization import BatchNormalization 
    from keras.utils import np_utils
    from keras.optimizers import SGD, Adam
    from keras.preprocessing.image import ImageDataGenerator
    import h5py
    import scipy
    import matplotlib.pyplot as plt


    #Params
    num_classes = 200
    batch_size = 128
    nb_epoch = 1

    """
    # load img, preprocessing, and save in a local h5 db

    path='./tiny-imagenet-200'
    X_train0,y_train,X_test0,y_test=load_images(path,num_classes)
    
    #keras2: img axes 64x64x3
    i=0
    X_train00 = np.zeros([num_classes*500,64,64,3],dtype='uint8')
    X_test00 = np.zeros([num_classes*50,64,64,3],dtype='uint8')
    for i in range(500):
        X_train00[0] = np.transpose(X_train0[i],(1,2,0))
        X_test00[0] = np.transpose(X_test0[i],(1,2,0))
    
    #resize
    new_shape = (224,224,3)
    X_train = np.empty(shape=(X_train00.shape[0],)+new_shape)
    for idx in range(X_train00.shape[0]):
        X_train[idx] = scipy.misc.imresize(X_train00[idx], new_shape)
        
    X_test = np.empty(shape=(X_test00.shape[0],)+new_shape)
    for idx in range(X_test00.shape[0]):
        X_test[idx] = scipy.misc.imresize(X_test00[idx], new_shape)    
     
    
    #print X dimension
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    
    num_samples=len(X_train)


    #convert data type and normalize values to range [0,1]
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255

    # convert 1-D class vectors to N-D class matrices
    Y_train = np_utils.to_categorical(y_train, num_classes)
    Y_test = np_utils.to_categorical(y_test, num_classes)
    
   
    #create h5
    h5f = h5py.File('IT.10.ad-20.classes.result.h5', 'w')
    h5f.create_dataset('xTrain', data=X_train)
    h5f.create_dataset('yTrain', data=Y_train)
    h5f.create_dataset('xTest', data=X_test)
    h5f.create_dataset('yTest', data=Y_test)
    
    h5f.close()
    
    print ('done')
    """
    ##################################################################
    
    #load from h5 db (already resized)
    h5f = h5py.File('IT.10.ad-200.vggs2.result.h5','r')#!!!
    X_train = h5f['xTrain'][:]
    Y_train = h5f['yTrain'][:]
    X_test =  h5f['xTest'][:]
    Y_test =  h5f['yTest'][:]
    
    h5f.close()
    print ('load')
    
      
    
    ##################################
    
    from keras.applications.resnet50 import ResNet50
    from keras.preprocessing import image
    from keras.applications.vgg16 import preprocess_input
    import numpy as np
    from keras.applications.inception_v3 import InceptionV3
    from keras.preprocessing import image
    from keras.models import Model
    from keras.layers import Dense, GlobalAveragePooling2D
    from keras import backend as K
    from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
    from keras.metrics import top_k_categorical_accuracy
    from keras.layers import Lambda, Input
    from keras.models import Model
    from keras.backend import tf as ktf

    #pretrain
    #inputs_tensor = (64, 64, 3)
    # 3 channel images of arbitrary shape
    inp = Input(shape=(64, 64, 3))
    try:
        resize = Lambda(lambda image: ktf.image.resize_images(image, (224, 224)))(inp)
    except :
    # if you have older version of tensorflow
        resize = Lambda(lambda image: ktf.image.resize_images(image, 224, 224))(inp)

    bmodel = ResNet50(input_tensor=resize, weights='imagenet', include_top=False, classes=200) # !! without weight and include_top = false will not work
    #re train
    #bmodel = ResNet50(include_top=False, classes=20)
    
    # add a global spatial average pooling layer
    x = bmodel.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(200, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(num_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=inp, outputs=predictions)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in bmodel.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.5, nesterov=True)
    adam = Adam(lr = 0.1 )
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    #data augm
    datagen = ImageDataGenerator(
                #zca_whitening = True,
                rotation_range=1,
                width_shift_range=0.2,
                height_shift_range=0.2,
                #shear_range=0.2,
                #zoom_range=0.2,
                #channel_shift_range=0.2,
                #cval=0.4, very cumbersome
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode='nearest'
              )
    datagen.fit(X_train)
    
    
    #checkpoint
    # checkpoint
    filepath="ep00.respre.ad-20.1.weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
    
    reduceLr = ReduceLROnPlateau(monitor='loss', patience=25, mode= 'auto', factor = 0.5)
    callbacks_list = [checkpoint, reduceLr]
    
    #fit model
    history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=nb_epoch, callbacks=callbacks_list,
                        validation_data=(X_test, Y_test))
    
    
    
    hist_acc = history.history['acc']
    hist_valAcc = history.history['val_acc']
    hist_loss = history.history['loss']
    hist_valLoss = history.history['val_loss']
    
    
    
    #create h5
    h5f = h5py.File('IT.20.ad-20.respre1.result.h5', 'w')
    h5f.create_dataset('acc', data=hist_acc)
    h5f.create_dataset('val_acc', data=hist_valAcc)
    h5f.create_dataset('loss', data=hist_loss)
    h5f.create_dataset('val_loss', data=hist_valLoss)
   
    
    
    h5f.close()
    
    #-------------plot-------------------
     #  "Accuracy"
    plt.plot(hist_acc )
    plt.plot(hist_valAcc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    # "Loss"
    plt.plot(hist_loss)
    plt.plot(hist_valLoss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
   

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


load
Epoch 1/1
780/781 [============================>.] - ETA: 0s - loss: 5.3308 - acc: 0.0083
Epoch 00001: val_acc improved from -inf to 0.00690, saving model to ep00.respre.ad-20.1.weights.best.hdf5
781/781 [==============================] - 492s 630ms/step - loss: 5.3307 - acc: 0.0083 - val_loss: 5.3688 - val_acc: 0.0069
